In [239]:
import pandas as pd

l1 = pd.read_csv(r"listings file 1 of 4.csv")
l2 = pd.read_csv(r"listings file 2 of 4.csv",names=l1.columns)
l3 = pd.read_csv(r"listings file 3 of 4.csv",names=l1.columns)
l4 = pd.read_csv(r"listings file 4 of 4.csv",names=l1.columns)


C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (43,88) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [240]:
conn = pd.concat([l1,l2,l3,l4])

In [245]:
conn['zipcode'].nunique()

246

In [229]:
ny = conn.copy()

In [230]:
#filter out for 2 bedroom properties
ny1 = ny[ny['bedrooms']==2]
ny1.shape

(4894, 95)

In [220]:
ny1.to_csv(r"2BHK-NY-Airbnb.csv",index=False)

In [231]:
#merge it with our zillow dataset
zw = pd.read_csv(r"ZillowData_NY.csv")
zw.head()


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04-01 00:00:00,1996-05-01 00:00:00,1996-06-01 00:00:00,...,2016-11-01 00:00:00,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,Current Selling Price,Rate
0,61639,10025,New York,NY,New York,New York,1,NaN,NaN,NaN,...,1366300,1354800.0,1327500,1317300,1333700,1352100,1390000,1431000,1431000,54760.0
1,61637,10023,New York,NY,New York,New York,3,NaN,NaN,NaN,...,1960900,1951300.0,1937800,1929800,1955000,2022400,2095000,2142300,2142300,73110.0
2,61703,10128,New York,NY,New York,New York,14,NaN,NaN,NaN,...,1527200,1541600.0,1557800,1582900,1598900,1646100,1720500,1787100,1787100,76660.0
3,61625,10011,New York,NY,New York,New York,15,NaN,NaN,NaN,...,2352200,2332100.0,2313300,2319600,2342100,2365900,2419700,2480400,2480400,96580.0
4,61617,10003,New York,NY,New York,New York,21,NaN,NaN,NaN,...,1937500,1935100.0,1915700,1916500,1965700,2045300,2109100,2147000,2147000,77930.0


In [232]:
zw1 = zw[['RegionName','State','Current Selling Price', 'Rate']]
zw1.columns =['zipcode','State','Current Selling Price', 'Rate']

In [233]:
zw1['zipcode'].nunique()

635

In [234]:
def mak(col):
    return str(col).strip()

zw1['zipcode'] = zw1['zipcode'].apply(mak)
ny1['zipcode'] = ny1['zipcode'].apply(mak)

C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [235]:
def getzip(col):
    return col[:5]
ny1['zipcode'] = ny1['zipcode'].apply(getzip)
zw1['zipcode'] = zw1['zipcode'].apply(getzip)


C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\nibed\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [236]:
#check how many zipcodes are common

need = zw1['zipcode'].values.tolist()
avl = ny1['zipcode'].values.tolist()

need = [int(i) for i in need]
avl = [int(i) for i in avl if i!='nan']
p = set(need).intersection(set(avl))
len(set(p))

22

In [237]:
mer = pd.merge(zw1,ny1,on='zipcode',how='left')
mer1 = mer[~mer['id'].isnull()]


In [238]:
#save the mergedataframe
mer1.to_csv(r"2BHK-NY-combined.csv",index=False)